In [1]:
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  

In [2]:
corpus=["I come to China to travel", 
        "This is a car polupar in China",          
        "I love tea and Apple ",   
        "The work is to write some papers in science"] 

vectorizer=CountVectorizer()

transformer = TfidfTransformer()

In [9]:
tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus)) 
#去除文章长度可能会产生的影响，除以文章词总个数
vectorizer = TfidfVectorizer(norm='l2', use_idf=False, sublinear_tf=True)

In [7]:
#一步完成向量化与tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer()
re = tfidf2.fit_transform(corpus)
print(re)

  (0, 16)	0.4424621378947393
  (0, 3)	0.348842231691988
  (0, 15)	0.697684463383976
  (0, 4)	0.4424621378947393
  (1, 5)	0.3574550433419527
  (1, 9)	0.45338639737285463
  (1, 2)	0.45338639737285463
  (1, 6)	0.3574550433419527
  (1, 14)	0.45338639737285463
  (1, 3)	0.3574550433419527
  (2, 1)	0.5
  (2, 0)	0.5
  (2, 12)	0.5
  (2, 7)	0.5
  (3, 10)	0.3565798233381452
  (3, 8)	0.3565798233381452
  (3, 11)	0.3565798233381452
  (3, 18)	0.3565798233381452
  (3, 17)	0.3565798233381452
  (3, 13)	0.3565798233381452
  (3, 5)	0.2811316284405006
  (3, 6)	0.2811316284405006
  (3, 15)	0.2811316284405006


In [12]:
from gensim.models import Word2Vec
# 引入数据集 
raw_sentences = ["the quick brown fox jumps over the lazy dogs", "yoyoyo you go home now to sleep"]
# 切分词汇,变为list of list格式 
sentences = [s.encode('utf-8').split() for s in sentences]
# 构建模型 
model = Word2Vec(sentences, min_count=1)

/Users/xuhan/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


NameError: name 'sentences' is not defined

In [14]:
from sklearn import datasets
twenty_train = datasets.load_files("./20news-bydate/20news-bydate-train")
twenty_test = datasets.load_files("./20news-bydate/20news-bydate-test")

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words="english",decode_error='ignore')  # 创建词频转换器
X_train_counts = count_vect.fit_transform(twenty_train.data)  # 转换训练集
X_train_counts.shape

(11314, 129782)

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 129782)

In [19]:
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [17]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_tfidf, twenty_train.target)
docs_new = ['God is love','OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)  #计算词频
X_new_tfidf = tfidf_transformer.transform(X_new_counts)  #计算TF-IDF
y_pred = clf.predict(X_new_tfidf)
for doc, category in zip(docs_new, y_pred):  #category是数字
    print(("%r => %s")%(doc, twenty_train.target_names[category])))
docs_new = ['God is love','OpenGL on the GPU is fast']

In [17]:
X_new_counts = count_vect.transform(docs_new)  #计算词频
X_new_tfidf = tfidf_transformer.transform(X_new_counts)  #计算TF-IDF
y_pred = clf.predict(X_new_tfidf)
for doc, category in zip(docs_new, y_pred):  #category是数字
    print(("%r => %s")%(doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [20]:
#管道流水化，对整个流程进行包装
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect',CountVectorizer(stop_words="english",decode_error='ignore')),
                    ('tfidf',TfidfTransformer()),
                    ('clf',MultinomialNB()),
                    ])
text_clf = text_clf.fit(twenty_train.data,twenty_train.target)  #训练集
text_clf.score(twenty_test.data, twenty_test.target)  #测试集

0.8169144981412639

In [21]:
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

text_clf_2 = Pipeline([('vect', CountVectorizer(stop_words='english',decode_error='ignore')), #去停用词
                      ('tfidf', TfidfTransformer()),
                      ('clf',SGDClassifier(loss = 'hinge', penalty = 'l2', alpha = 1e-3, max_iter = 5, random_state = 42)),
                      ])
text_clf_2.fit(twenty_train.data, twenty_train.target)
# text_clf_2.score(twenty_test.data, twenty_test.target)（一样的效果）
y_pred = text_clf_2.predict(twenty_test.data)
metrics.accuracy_score(twenty_test.target, y_pred)

/Users/xuhan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


0.8227562400424854

In [22]:
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

text_clf_2 = Pipeline([('vect', CountVectorizer(stop_words='english',decode_error='ignore')), #去停用词
                      ('tfidf', TfidfTransformer()),
                      ('clf',SGDClassifier(loss = 'hinge', penalty = 'l2', alpha = 1e-3, max_iter = 5, random_state = 42)),
                      ])
text_clf_2.fit(twenty_train.data, twenty_train.target)
text_clf_2.score(twenty_test.data, twenty_test.target)

/Users/xuhan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


0.8227562400424854

In [23]:
print(metrics.classification_report(twenty_test.target, y_pred, target_names = twenty_test.target_names))  # 要加print

                          precision    recall  f1-score   support

             alt.atheism       0.73      0.70      0.71       319
           comp.graphics       0.80      0.69      0.74       389
 comp.os.ms-windows.misc       0.73      0.77      0.75       394
comp.sys.ibm.pc.hardware       0.71      0.67      0.69       392
   comp.sys.mac.hardware       0.81      0.82      0.82       385
          comp.windows.x       0.85      0.77      0.81       395
            misc.forsale       0.81      0.87      0.84       390
               rec.autos       0.91      0.90      0.90       396
         rec.motorcycles       0.93      0.96      0.95       398
      rec.sport.baseball       0.88      0.92      0.90       397
        rec.sport.hockey       0.87      0.98      0.93       399
               sci.crypt       0.86      0.96      0.91       396
         sci.electronics       0.80      0.63      0.71       393
                 sci.med       0.90      0.86      0.88       396
         

In [24]:
print(metrics.confusion_matrix(twenty_test.target, y_pred))

[[223   1   0   1   0   0   2   0   2   3   0   2   1   9   6  48   2   5
    1  13]
 [  2 269  20   9   9  26   3   1   4   9   4   8   5   1   9   2   2   3
    1   2]
 [  0  10 303  24  12  11   1   3   1   5   3   7   2   1   6   1   0   1
    0   3]
 [  3   8  30 261  21   4  18   3   3   3   2   3  21   1   5   0   2   2
    1   1]
 [  1   4   8  24 316   1  11   0   1   4   1   1   6   2   1   0   2   1
    1   0]
 [  1  28  41   0   3 304   3   0   0   1   1   2   1   1   7   1   1   0
    0   0]
 [  0   2   0  17   6   0 340   8   1   2   3   1   3   2   3   0   1   1
    0   0]
 [  1   1   1   2   1   0  10 355   6   1   0   0  10   1   2   0   4   0
    1   0]
 [  0   0   0   1   0   0   3   5 384   2   0   0   1   1   0   0   0   0
    0   1]
 [  0   0   0   0   1   0   3   0   0 365  27   0   0   0   0   0   1   0
    0   0]
 [  0   0   0   0   1   0   0   0   0   3 393   0   0   0   0   2   0   0
    0   0]
 [  0   1   1   0   2   0   3   3   0   0   1 380   3   0   0   0

In [25]:
from sklearn.model_selection import GridSearchCV
parameters = {
            'vect__ngram_range':[(1,1), (1,2)],
             'tfidf__use_idf':(True, False),
             'clf__alpha':(1e-2, 1e-3)
             }
gs_clf = GridSearchCV(text_clf_2, parameters, n_jobs = -1, cv=5)  # text_clf_2: SVC的pipeline,n_jobs=-1, 5折交叉验证
grid_result = gs_clf.fit(twenty_train.data, twenty_train.target)
print("Best: %f \nusing %s" % (grid_result.best_score_, grid_result.best_params_)) # best_params_：<class 'dict'>

Best: 0.906134 
using {'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


/Users/xuhan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


In [26]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
parameters = {
            'vect__ngram_range':[(1,1),(1,2),(2,2)],
             'tfidf__use_idf':(True,False),
             'clf__alpha':(1e-1,1e-2,1e-3,1e-4,1e-5)
             }
rs_clf = RandomizedSearchCV(text_clf_2, parameters,n_jobs = -1, cv=5)
rs_result = rs_clf.fit(twenty_train.data, twenty_train.target)
print("Best: %f \nusing %s" % (rs_result.best_score_, rs_result.best_params_))

Best: 0.929115 
using {'vect__ngram_range': (1, 2), 'tfidf__use_idf': True, 'clf__alpha': 0.0001}


/Users/xuhan/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf_3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()), 
]) 

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 10000, 50000,60000),  #
    'vect__ngram_range': [(1, 1), (1, 2),(2,2)],  
    'tfidf__norm': ('l1', 'l2'), #
    'clf__alpha': (0.0001, 0.00001, 0.000001,0.0000001),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__max_iter': (10, 50, 80),  #
}

gs_clf = GridSearchCV(text_clf_3, parameters, n_jobs = -1, cv=5) # text_clf_2: SVC的pipeline,n_jobs=-1，5折交叉验证
grid_result = gs_clf.fit(twenty_train.data, twenty_train.target)
print("Best: %f \nusing %s" % (grid_result.best_score_, grid_result.best_params_))

/Users/xuhan/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/xuhan/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb5 in position 894: invalid start byte